In [2]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import time
from jupyter_util import *
from importlib import reload


In [3]:
def assert_success(d):
    if isinstance(d, dict) is False:
        print("not dict", type(d))
    if d.get('success') is not True:
        print(d)

In [4]:
# test_pds_return_same_id()
set_pds_config("local")
save_and_retrieve("local")
set_pds_config("data_vault", {"api_url": "http://ocadatavault"})
save_and_retrieve("data_vault_thcf")
set_pds_config("data_vault", {"api_url": "http://ocadatavault"}, "vault2")
save_and_retrieve("data_vault_thcf_2")
set_own_your_data()
payload_id = save_and_retrieve("own_your_data")
## TEST that Settings are persistent! 
activate_pds_both_agents("data_vault")
save_and_retrieve("data_vault_thcf")
activate_pds_both_agents("data_vault", "vault2")
save_and_retrieve("data_vault_thcf_2")
activate_pds_both_agents("own_your_data")
payload_id = save_and_retrieve("own_your_data")


500 Internal Server Error

Server got itself in trouble
status_code: 500  value: 500 Internal Server Error

Server got itself in trouble


AssertionError: status_code is not 200

In [5]:
def test_invalid_own_your_data():
    set_pds_config(
        "own_your_data",
        [
  {
    "instance_name": "string",
    "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE",
    "driver": {
      "own_your_data": {
        "grant_type": "client_credentials",
        "api_url": "https://data-vault.eu"
      },
      "thcf_data_vault": {
        "host": "string"
      },
      "name": "own_your_data"
    },
    "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
  }
],
    )
test_invalid_own_your_data()

AttributeError: 'list' object has no attribute 'update'

In [ ]:
[
  {
    "instance_name": "string",
    "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE",
    "driver": {
      "own_your_data": {
        "grant_type": "client_credentials",
        "api_url": "https://data-vault.eu"
      },
      "thcf_data_vault": {
        "host": "string"
      },
      "name": "own_your_data"
    },
    "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
  }
]

In [51]:
def set_data_vault():
    set_pds_config("data_vault", {"api_url": "http://ocadatavault"})
    
def set_own_your_data():
    set_pds_config(
        "own_your_data",
        {
            "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE",
            "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg",
            "api_url": "https://data-vault.eu",
        },
    )

# set_data_vault()
set_own_your_data()

{"own_your_data": {"connected": true}}


In [52]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

agent1 = processInviteUrl(agent1['url'])
agent2 = processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

agent1['connection'] = connectWithAcapy(agent1, plugin)
agent2['connection'] = connectWithAcapy(agent2, plugin)

agent1['connection_details'] = decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = decodeConnectionDetails(agent2['connection'])
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

connection = connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMTk0MmQzMzAtMmE2Zi00NmQyLWI2NDktYTkwZDJiM2E5N2Y2IiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyJITVRIdER5ajdpdG5ZTnhaR3NYOFR5NnVUZVI5YmJoYVFiazRZYTVWNHR3USJdLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9hZ2VudDEubG9jYWxob3N0In0=
http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNTU3N2ZmM2YtN2U0MS00MjY2LWExMDYtZTJkNWY4NTgzNDVjIiwgInJlY2lwaWVudEtleXMiOiBbIkRmWUVZdnp2MWs2ZkN6aVQ1eEJHbzJHd2l3bUtSckNBZHczYlhKTEVlQUFIIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QiLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==
Decoded invitation:

{'@id': '5577ff3f-7e41-4266-a106-e2d5f858345c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['DfYEYvzv1k6fCziT5xBGo2GwiwmKRrCAdw3bXJLEeAAH'],
 

In [53]:
## TEST PDS
# time.sleep(0.2)

# add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={1234}')
# print(add.text)

# set_pds_config("own_your_data", {
#     "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
#     "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
# })
# payload_id = save_and_retrieve("own_your_data1234")
# add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={payload_id}')
# assert '{"message_sent": "success"}' == add.text

# time.sleep(3)
# get1 = requests.get('http://localhost:8150/pds/' + str(payload_id))
# get2 = requests.get('http://localhost:8151/pds/' + str(payload_id))
# print(get1.text, get2.text)
# assert get1.status_code == get2.status_code, f"status codes are not equal {get1.status_code, get2.status_code}"
# assert get1.text == get2.text, "exchanged data with get_from is not equal"
# set_pds_config("data_vault", {"host": "http://ocadatavault"})

In [54]:
time.sleep(0.2)

add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={1234}')
print(add.text)

set_own_your_data()
payload_id = save_and_retrieve("own_your_data1234")
add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={payload_id}')

{"success": true}
{"own_your_data": {"connected": true}}


In [55]:
time.sleep(0.5)
newDid1 = createDid(agents[1])
newDid0 = createDid(agents[0])

try:
    registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

setPublicDid(agents[1], newDid1['result']['did'])
setPublicDid(agents[0], newDid0['result']['did'])
time.sleep(0.5)

In [56]:
set_pds_config_agent1(
    "own_your_data",
    {
        "client_id": "-s2bdkM_cv7KYDF5xg_Lj6vil1ZJaLQJ79duOW7J9g4",
        "client_secret": "s_dR8dzbVES_vvc1-nyb1O_cuzyCz2_bRd3Lr12s4ug",
        "api_url": "https://data-vault.eu",
    },
)

set_pds_config_agent2(
    "own_your_data",
    { "scope": "admin", "client_id": "50c67447e28eeb5ac56bd8bc3fbfbc5916f817603eb3f117f01e84d52e36afb7", "client_secret": "6f20318f0a451b24a9c4aaa00214626d6e0f6b15d18957acd11a085c4d7b2dbd", "api_url": "https://dip-officer.data-container.net" }
)


issue_cred_request = post(8150, '/issue-credential/request', {
  "credential_values": {
    "type": "Person",
    "address": {
      "type": "PostalAddress",
      "streetAddress": "123 Main St.",
      "addressLocality": "Blacksburg",
      "addressRegion": "VA",
      "postalCode": "24060",
      "addressCountry": "US",
    }
  },
  "connection_id": connection[0],
})
assert_success(issue_cred_request)

{"own_your_data": {"connected": true}}
{"own_your_data": {"connected": true}}


In [57]:
cred_exchange = get(8151, f"/issue-credential/exchange/record?connection_id={connection[1]}&thread_id={issue_cred_request['thread_id']}")
assert_success(cred_exchange)
cred_exchange = cred_exchange['result']

In [58]:
id = cred_exchange[0]['credential_exchange_id']
issue_cred = post(8151, "/issue-credential/issue?credential_exchange_id=" + id)
assert_success(issue_cred)

In [59]:
# REQUEST PROOF (2)
request_proof = post(8151, '/present-proof/request', {
  "connection_id": connection[1],
  "schema_base_dri": "12345",
  "requested_attributes": ["address"],
  "issuer_did": "123145"
})
assert_success(request_proof)
exchange_id_agent2 = request_proof['exchange_id']

In [60]:
# FETCH EXCHANGE RECORDS (1)
exchange_record = get(8150, f"/present-proof/exchange/record?state=request_received&connection_id={connection[0]}&thread_id={request_proof['thread_id']}")
assert_success(exchange_record)

# FETCH CREDENTIALS (1)
creds = get(8150, "/credentials")
assert_success(creds)

# PRESENT PROOF (1)
present = post(8150, '/present-proof/present', {
  "credential_id": creds['result'][-1]['dri'],
  "exchange_record_id": exchange_record['result'][0]['presentation_exchange_id']
})
assert_success(present)

In [61]:
time.sleep(0.5)
issue_cred = post(8151, f'/present-proof/acknowledge?status=true&exchange_record_id={exchange_id_agent2}')
assert_success(issue_cred)

# Services

In [62]:
import random
add_consent = post(8151, "/verifiable-services/consents", {
    "label": "test_" + str(random.randint(1, 100000000)),
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "oca_data": {
        "expiration": "7200",
        "limitation": "7200",
        "dictatedBy": "test",
        "validityTTL": "7200",
    }
})
assert_success(add_consent)
get_consents_provider = get(8151, "/verifiable-services/consents")
assert_success(get_consents_provider)
get_consents_user = get(8150, "/verifiable-services/consents")
assert_success(get_consents_user)
# assert len(get_consents_user['result']) == 0
# get(8150, "/verifiable-services/given-consents")

In [63]:
add = post(8151, '/verifiable-services/add', {
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "12345"
  },
  "consent_id": add_consent['consent_id']
})
assert_success(add)


In [64]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
service_list = json.loads(service_list.text)
webhook_service_list = requests.get('http://localhost:8150/verifiable-services/request-service-list/' + connection[0])
webhook_service_list = json.loads(webhook_service_list.text)
assert_success(service_list)
assert_success(webhook_service_list)


# print(service_list['services'][0])
# assert service_list['services'][0]['consent_schema']['usage_policy'].find("DataControllerPolicy") != -1, "consent schema in service should have usage policy for data controller"

In [65]:
service = service_list['result']['services'][0]
service.pop("policy_validation", None)
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "user_data": "abckonoin",
  "service": service,
})
apply = json.loads(apply.text)
assert_success(apply)

In [69]:
time.sleep(0.5)
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
assert_success(issue_self)
issue_self = issue_self['result']

In [70]:
time.sleep(0.5)
process_application = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
process_application = json.loads(process_application.text)
assert_success(process_application)

In [71]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
assert_success(credentials)
pp.pprint(credentials)

{'result': [{'content': {'context': ['https://www.w3.org/2018/credentials/v1',
                                     'https://www.schema.org'],
                         'credentialSubject': {'address': {'addressCountry': 'US',
                                                           'addressLocality': 'Blacksburg',
                                                           'addressRegion': 'VA',
                                                           'postalCode': '24060',
                                                           'streetAddress': '123 '
                                                                            'Main '
                                                                            'St.',
                                                           'type': 'PostalAddress'},
                                               'subject_id': 'Hw3iKVUWtsi73WUGacxiY1',
                                               'type': 'Person'},
                         'issu

In [36]:
save_data_to_pds = post(8151, "/pds/save", {"payload": "asfag"})
pds_get_info_from = post(8150, f"/pds/get_from?connection_id={connection[0]}&payload_id={save_data_to_pds['payload_id']}")
print(pds_get_info_from)

{'success': True}
